# 파일 불러오기 및 데이터 확인

In [86]:
import pandas as pd

PATH = "accounting_dataset.csv"
doc = pd.read_csv(PATH)

doc.head()

,Transaction_ID,Date,Account_Number,Transaction_Type,Amount,Currency,Counterparty,Category,Payment_Method,Risk_Incident,Risk_Type,Incident_Severity,Error_Code,User_ID,System_Latency,Login_Frequency,Failed_Attempts,IP_Region
0,TXN00001,2024-08-18,250637,Refund,952.11,USD,Garcia-Gordon,Payroll,Cash,0,NaN,NaN,NaN,U001,299.26,6,2,TH
1,TXN00002,2024-11-26,122794,Debit,3293.29,USD,"Heath, Pena and Buchanan",Payroll,Cash,1,Error,Low,E002,U233,248.61,4,1,TW
2,TXN00003,2024-04-30,152231,Debit,2849.97,USD,Pham PLC,Operations,Cash,0,NaN,NaN,NaN,U251,378.61,6,2,BI
3,TXN00004,2024-03-23,862833,Refund,9813.53,USD,"Levine, Long and Stewart",Payroll,Cash,0,NaN,NaN,NaN,U235,242.19,9,3,LV
4,TXN00005,2024-07-03,436506,Credit,1228.69,USD,Lee LLC,Inventory,Bank Transfer,0,NaN,NaN,NaN,U500,327.81,4,1,DZ


# 프로젝트 개괄
## 부정회계탐지하여 금융거래의 신뢰성 향상
### 거래금액(Amount), Login_Frequency과의 관계성을 보고 어떤 요인이 Risk Incident에 큰 영향을 주는지 보려고 함

### 부정회계에 영향을 주는 요소들을 파악하고, 사용자가 시스템 이용시, UI로 경고메시지를 추가해주거나, 직접 모니터링을 통해 부정회계를 방지할 수 있음.

### 1-1) EDA를 기반으로 한 데이터 전처리

In [7]:
doc.shape

(10000, 18)

In [87]:
doc.describe()

,Account_Number,Amount,Risk_Incident,System_Latency,Login_Frequency,Failed_Attempts
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,550262.371600,4988.750157,0.144800,300.978158,5.494700,2.490900
std,260526.980007,2890.160301,0.351917,115.005259,2.870783,1.707164
min,100018.000000,12.130000,0.000000,100.030000,1.000000,0.000000
25%,322302.500000,2449.010000,0.000000,201.925000,3.000000,1.000000
50%,551715.000000,4974.745000,0.000000,301.690000,5.000000,2.000000
75%,772783.750000,7481.250000,0.000000,400.385000,8.000000,4.000000
max,999946.000000,9999.540000,1.000000,499.990000,10.000000,5.000000


In [9]:
doc.columns

Index(['Transaction_ID', 'Date', 'Account_Number', 'Transaction_Type',
       'Amount', 'Currency', 'Counterparty', 'Category', 'Payment_Method',
       'Risk_Incident', 'Risk_Type', 'Incident_Severity', 'Error_Code',
       'User_ID', 'System_Latency', 'Login_Frequency', 'Failed_Attempts',
       'IP_Region'],
      dtype='object')

## Merge에 사용될 Transaction_ID와 각 변수의 데이터 전처리 및 개별 CSV 파일화

In [15]:
amount = doc[["Transaction_ID", "Amount"]]
amount.isnull().sum() 
amount.head()
amount.to_csv("amount.csv", index=False) 

,Transaction_ID,Amount
0,TXN00001,952.11
1,TXN00002,3293.29
2,TXN00003,2849.97
3,TXN00004,9813.53
4,TXN00005,1228.69


In [27]:
login_frequency = doc[["Transaction_ID", "Login_Frequency"]]
login_frequency.isnull().sum() 
login_frequency.head()
login_frequency.to_csv("login_frequency.csv", index=False) 

In [39]:
risk_incident = doc[["Transaction_ID", "Risk_Incident"]]
risk_incident.isnull().sum() 
risk_incident.head()
risk_incident.to_csv("risk_incident.csv", index=False)

In [83]:
risk_type.loc[:,"Risk_Type"] = risk_type["Risk_Type"].astype("object").fillna("None") # .loc를 통해서 데이터프레임의 진짜열을 지정해주기
risk_type.head() # 결측치를 None로 바꿔주고, 데이터타입 object로 변환
risk_type["Risk_Type"].dtype # Object
risk_type.to_csv("risk_type.csv", index=False) # index제외하고 risk_type만 저장

### Merge를 위한 중복값 확인

In [62]:
amount.duplicated() # Nothing

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Length: 10000, dtype: bool

In [65]:
login_frequency.duplicated() # Nothing

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Length: 10000, dtype: bool

In [67]:
risk_incidents.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Length: 10000, dtype: bool

In [82]:
risk_type.duplicated() # 10001 value, which is unnecessary. REMOVE
#risk_type = risk_type.drop(["Risk_Type"])
#risk_type.tail()


0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Length: 10000, dtype: bool